# Zygote
Date: 22.8.2021

- Juliaの自動微分のライブラリ．

## 自動微分(Automatic differentiation)とは
> 自動微分（じどうびぶん、アルゴリズム的微分とも）とは、プログラムで定義された関数を解析し、偏導関数の値を計算するプログラムを導出する技術である。

詳細はwiki：https://ja.wikipedia.org/wiki/%E8%87%AA%E5%8B%95%E5%BE%AE%E5%88%86

In [1]:
using Zygote
using Pkg

In [2]:
Pkg.status()

      Status `~/Dropbox/01code/julia/intro_julia/Zygote/Project.toml`
  [e88e6eb3] Zygote v0.6.20


In [3]:
VERSION

v"1.6.0"

## 一変数関数の微分

In [4]:
f(x) = 5x + 3

f (generic function with 1 method)

In [5]:
f(10), f'(10)

(53, 5.0)

In [6]:
@code_llvm f'(10)
#^ これは何をしているのだろうか？

;  @ /Users/siida/.julia/packages/Zygote/nsu1Y/src/compiler/interface.jl:79 within `#52'
define double @"julia_#52_4146"(i64 signext %0) {
top:
  ret double 5.000000e+00
}


## Zygote.gradientの使用
- 三角関数を格納した辞書を作成
- それぞれのkeyを`readline()`で読み込む
- 読み込んだ関数に対してAutomatic differentiation (AD)（`gradient`）を行う．
- $\pi$ を代入する

In [7]:
fs = Dict("sin" => sin, "cos" => cos, "tan" => tan)
Zygote.gradient(x -> fs[readline()](x), π)

stdin>  sin


(-1.0,)

Note) このような柔軟性を持っている．またこのように書いてもパフォーマンスは落ちないらしい．

## 自分でカスタムした`gradient`を作るのは簡単
Zygoteを拡張する！そのためには`@adjoint`を使う．
うーん．挙動がよくわかってない．

参考：
- https://fluxml.ai/Zygote.jl/latest/adjoints/#Custom-Adjoints-2

In [8]:
using Zygote: @adjoint 
add(a, b) = a + b #追加したい関数の定義
@adjoint add(a, b) = add(a, b), Δ -> (Δ, Δ) 
# ^ Zygote.gradientで使用可能な関数を拡張
# add(a, b)を

In [9]:
Zygote.gradient(add, 1, 2)

(1, 1)

## Zygoteはimplicitly-used parametersを微分できる

In [10]:
W, b = rand(2, 3), rand(2)
predict(x) = W*x .+ b
g = Zygote.gradient(Params([W, b])) do 
    sum(predict([1,2,3]))
end

Grads(...)

In [11]:
g[W], g[b]

([1.0 2.0 3.0; 1.0 2.0 3.0], Fill(1.0, 2))

In [12]:
# g is equivalent to 
gs = gradient(() -> sum(predict([1,2,3])), Params([W, b]))

Grads(...)

In [13]:
gs[W], gs[b]

([1.0 2.0 3.0; 1.0 2.0 3.0], Fill(1.0, 2))